In [1]:
import pandas as pd
import numpy as np

## Intersección de usuarios (**con imágenes**) entre ciudades para domain adaptation

In [2]:
cities = ["Gijon", "Barcelona", "Warsaw", "Budapest", "Hamburg", "Vienna", "Bucharest", "New York", "Paris", "Rome", "Madrid", "Berlin", "London"]

cities_data = {}

print(f"{'':10s}\titems\treviews\tusers")

for city in cities:
    city = city.lower().replace(" ", "")
    city_path = f"out/{city}"
    category_path = f"{city_path}/restaurants"

    item_path = f"{category_path}/items.pkl"
    review_path = f"{category_path}/reviews.pkl"
    user_path = f"{category_path}/users.pkl"

    items = pd.read_pickle(item_path)
    reviews = pd.read_pickle(review_path)
    users = pd.read_pickle(user_path)

    reviews["n_images"] = reviews["images"].apply(lambda x: len(x))
    users_with_images = reviews.loc[reviews.n_images>0]["userId"].unique()

    cities_data[city]={"items":items, "reviews":reviews, "users":users, "users_images":users_with_images}

    print(f"{city:10s}\t{len(items)}\t{len(reviews)}\t{len(users)}")

          	items	reviews	users
gijon     	840	80753	41980
barcelona 	9523	1438248	771555
warsaw    	3606	197274	108006
budapest  	3869	565005	330198
hamburg   	3287	190865	102362
vienna    	4801	490952	250106
bucharest 	2614	143560	82002
newyork   	10129	1256917	572848
paris     	16937	2126379	1029499
rome      	11649	2887380	1334951
madrid    	11456	1348056	625869
berlin    	7452	574100	309859
london    	21266	2956339	1455474


In [3]:
city_names = "\t".join(cities_data.keys())
print(f'city\t{city_names}')
      
for c_urs in cities_data.keys():
  print(c_urs, end="\t")
  for o_urs in cities_data.keys():
    c_data = set(cities_data[c_urs]["users_images"])
    o_data = set(cities_data[o_urs]["users_images"])
    print(len(c_data.intersection(o_data)), end="\t")
  print()

city	gijon	barcelona	warsaw	budapest	hamburg	vienna	bucharest	newyork	paris	rome	madrid	berlin	london
gijon	10070	661	51	157	26	131	32	182	252	407	1911	165	266	
barcelona	661	178220	1293	3868	1101	3489	863	5251	9565	10658	11361	3955	8907	
warsaw	51	1293	25988	1422	308	1093	521	736	1238	1661	1082	1243	1507	
budapest	157	3868	1422	81719	674	5253	1102	2318	3978	5365	3038	3028	4834	
hamburg	26	1101	308	674	20043	1169	161	778	1153	1019	673	2402	1356	
vienna	131	3489	1093	5253	1169	55469	800	2329	4126	4874	2780	3349	4309	
bucharest	32	863	521	1102	161	800	20430	498	915	1411	754	651	1091	
newyork	182	5251	736	2318	778	2329	498	98832	7510	6527	4114	2797	9036	
paris	252	9565	1238	3978	1153	4126	915	7510	184542	11711	6298	4355	12981	
rome	407	10658	1661	5365	1019	4874	1411	6527	11711	281425	7835	4736	10619	
madrid	1911	11361	1082	3038	673	2780	754	4114	6298	7835	139702	3099	6073	
berlin	165	3955	1243	3028	2402	3349	651	2797	4355	4736	3099	62240	5122	
london	266	8907	1507	4834	1356	4309	1091	9036

In [ ]:
data_path = "/content/drive/Shareddrives/DataSets/TripAdvisor/"

usrs_gjn = set(pd.read_pickle(f"{data_path}gijon/users.pkl")["id"].unique())
usrs_bcn = set(pd.read_pickle(f"{data_path}barcelona/users.pkl")["id"].unique())
usrs_mdr = set(pd.read_pickle(f"{data_path}madrid/users.pkl")["id"].unique())
usrs_prs = set(pd.read_pickle(f"{data_path}paris/users.pkl")["id"].unique())
usrs_nyc = set(pd.read_pickle(f"{data_path}newyorkcity/users.pkl")["id"].unique())
usrs_lnd = set(pd.read_pickle(f"{data_path}london/users.pkl")["id"].unique())

print(f"{len(usrs_gjn)}\t{len(usrs_bcn)}\t{len(usrs_mdr)}\t{len(usrs_nyc)}\t{len(usrs_prs)}\t{len(usrs_lnd)}")

26450	184307	246618	430083	463097	1037844


In [ ]:
users = [usrs_gjn, usrs_bcn, usrs_mdr, usrs_prs, usrs_nyc, usrs_lnd]

for c_urs in users:
  for o_urs in users:
    print(len(c_urs.intersection(o_urs)), end="\t")
  print()

26450	2131	6298	20	52	93	
2131	184307	22760	307	708	1276	
6298	22760	246618	292	984	1641	
20	307	292	463097	1685	3251	
52	708	984	1685	430083	59238	
93	1276	1641	3251	59238	1037844	


|         | **GJN** | **BCN** | **MDR** | **NYC** | **PRS** | **LND** |
|---------|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|
| **GJN** |  *26450*  |   2131  |   6298  |    20   |    52   |    93   |
| **BCN** | -    | *184307*  | 22760   | 307     | 708     | 1276    |
| **MDR** | -    | -   | *246618*  | 292     | 984     | 1641    |
| **NYC** | -      | -     | -     | *463097*  | 1685    | 3251    |
| **PRS** | -      | -     | -     | -    | *430083*  | 59238   |
| **LND** | -      | -    | -    | -    | -   | *1037844* |

In [ ]:
def fotos_por_restaurante(reviews, city):
  data = reviews.groupby("restaurantId").n_img.sum().reset_index()
  histo = sbs.histplot(data, x="n_img", kde=False, bins=range(1,22,1));
  histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
  labels = list(map(str,range(1,20)))+["≥20"]
  histo.set_xticklabels(labels)
  histo.set_xlabel("Número de fotos")
  histo.set_ylabel("Número de restaurantes")
  plt.tight_layout()
  histo.figure.savefig(f"{city}_hist_fotos_pr_rst.pdf")
  plt.clf()
  
def fotos_por_review(reviews, city):
  return reviews
def fotos_por_usuario(reviews, city):
  return reviews
def reviews_por_restaurante(reviews, city):
  data = reviews.groupby("restaurantId").reviewId.count().reset_index()
  data.loc[data.reviewId>=20, "reviewId"]=20

  histo = sbs.histplot(data, x="reviewId", kde=False, bins=range(1,22,1));
  histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
  labels = list(map(str,range(1,20)))+["≥20"]
  histo.set_xticklabels(labels)
  histo.set_xlabel("Número de reseñas")
  histo.set_ylabel("Número de restaurantes")
  plt.tight_layout()
  histo.figure.savefig(f"{city}_hist_rvws_pr_rst.pdf")

def reviews_por_usuario(reviews, city):
  return reviews

In [ ]:
cities = ["gijon", "barcelona", "paris", "madrid", "newyorkcity", "london"]
# cities = ["gijon"]

base_path = "/content/drive/Shareddrives/Media"

for city in cities:
  data_path = f"/content/drive/Shareddrives/DataSets/TripAdvisor/{city}/"
  reviews = pd.read_pickle(data_path+"reviews.pkl")

  # reviews["n_img"] = reviews.images.apply(lambda x: len(x))
  restaurants = pd.read_pickle(data_path+"restaurants.pkl")
  users = pd.read_pickle(data_path+"users.pkl")
  
  # dates = pd.to_datetime(reviews.date, format="%d de %B de %Y" )
  # print(dates.min(), dates.max())

  # n_images  = reviews.n_img.sum()
  # dt = [city, len(reviews), len(users), len(restaurants), n_images]
  # print("\\"+" & ".join(map(str, dt))+" \\\\")

  reviews = reviews[["date", "images", "language", "rating", "restaurantId", "reviewId", "text", "title", "url", "userId"]]
  users = users[["id", "name", "location"]]
  restaurants = restaurants[["id", "name", "city", "priceInterval", "url", "rating"]]

  reviews.iloc[:50].to_csv(f"{base_path}/{city}_revs.csv", encoding="utf-8")
  users.iloc[:50].to_csv(f"{base_path}/{city}_usrs.csv", encoding="utf-8")
  restaurants.iloc[:50].to_csv(f"{base_path}/{city}_rsts.csv", encoding="utf-8")

In [ ]:
reviews["text"] = reviews.text.str.lower().str.split(" ")

In [ ]:
reviews.text.str.lower().str.contains("definitivas").value_counts()

AttributeError: ignored

In [ ]:
cities = {"gijon":0.3}#, "barcelona":1.6, "madrid":2.1, "newyorkcity":8.3, "paris":8.9, "london":3.2} # Ciudades con su población
stats = []

for city in cities.keys():
  data_path = f"/content/drive/Shareddrives/DataSets/TripAdvisor/{city}/"

  reviews = pd.read_pickle(data_path+"reviews.pkl")
  # restaurants = pd.read_pickle(data_path+"restaurants.pkl")
  # users = pd.read_pickle(data_path+"users.pkl")

  reviews["n_img"] = reviews.images.apply(lambda x: len(x))

  n_reviews = len(reviews.reviewId.unique())
  n_images  = reviews.n_img.sum()
  n_users = len(reviews.userId.unique())
  n_rest = len(reviews.restaurantId.unique())

  stats.append((f"\{city}", cities[city], n_reviews, n_images, n_users, n_rest))

  fotos_por_restaurante(reviews, data_path+city)
  reviews_por_restaurante(reviews, data_path+city)

stats = pd.DataFrame(stats, columns=["Conjunto", "Población", "Reseñas", "Imágenes", "Usuarios", "Restaurantes"])
print(stats.to_latex(index=False, escape=False))

In [ ]:
data = reviews.groupby("restaurantId").n_img.sum().reset_index()
data.loc[data.n_img>=20, "n_img"]=20

histo = sbs.histplot(data, x="n_img", kde=False, bins=range(1,22,1));
histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
labels = list(map(str,range(1,20)))+["≥20"]
histo.set_xticklabels(labels)
histo.set_xlabel("Número de fotos")
histo.set_ylabel("Número de restaurantes")
plt.tight_layout()


In [ ]:
# Datos para Bea (con menos columnas)
data_sm = reviews[["reviewId", "userId", "restaurantId", "date", "images", "rating", "url"]]
data_sm.to_pickle(data_path+"reviews_sm.pkl")
